# CaBER Video Processing

In [ ]:
import sys
sys.path.append('../')
import time
import numpy as np

In [ ]:
from video import Video # In your own code, change to: "from caber_image_processing import Video"

In this example script, the processing of a video recorded CaBER experiment is shown. A video is processed just like an image sequence. When the Video object is created, a temporary folder of images is populated and frames are processed individually. The Video class is essentially a wrapper for the ImageSequence class. To begin, designate a path to the video which the user would like to analyze.

In [ ]:
video_path = '../data/Stock-PG0d4_2000fps_1.mp4'

Video objects have similar configuration parameters to the ImageSequence and Image classes. In this example we will explore just a few options. Detailed explanation of other parameters can be found in the library's documentation.

The playback FPS must be supplied with the video path to properly extract frames. If the playback FPS is not the same as the time recorded between frames, time_btwn_frames must be supplied in seconds as well to correctly space out radius measurements in time. By default, the user will be prompted to crop their image to the exact width the fluid. The width of this crop should be provided in meters (all inputs should be in SI units). The degree of the polynomial edge approximation can also be changed but is defaulted to 4. To create this polynomial, only the middle 40% of the edge points is used. This percentage can be expanded too.

In [ ]:
playback_fps = 30 # Standard frames per second
time_btwn_frames = 0.149*(10**(-3)) # In seconds (different from 1/FPS because video plays back slower than it was recorded)
crop_width = 6*(10**(-3)) # In meters
polynomial_degree = 4 # For edge approximation
vertical_percentage_to_consider = 0.4 # Middle percentage of picture height to consider, should contain the necessary part of the fluid (where a minimum radius may be)

Crop window will show up on creation of Video object, select a window such to contain all the fluid. The horizontal width of the image should match the initial diameter of the fluid. This will be used in tandem with the crop_width parameter to convert between integer pixel width and real-world diameter in meters.

Each frame of the video is processed simultaneously, each as a single image. This is done to speed processing down to a matter of seconds.

In [ ]:
# Construct video object and get radii
start = time.time()
vid = Video(video_path, playback_fps, time_btwn_frames=time_btwn_frames, width=crop_width, poly_degree=polynomial_degree, pct_considered=vertical_percentage_to_consider, graph_title='Stock-PG0d4_2000fps_1.mp4')
end = time.time()
print('Analyzed ' + str(len(vid.images)) + ' frames in ' + str(end - start) + ' seconds')

In [ ]:
# Plot data over time
vid.plot_radius()

As you can see, the radius decreases with time until reaching a breaking point where the radius is zero. This data can be fit to an equation to gain information about the behavior of the fluid. To begin, choose the time range over which the fit should be calculated. By default, if no start or end time is supplied, an equation will be fit over the entire range of the experiment. Here, the approximately linear middle portion is chosen to try and get an accurate fit.

In [ ]:
time_start = 0.025 # Where to start the fit domain
time_end = 0.25 # Where to end the fit domain

To fit, initial conditions are needed. Most notably, the radius at the start time must be found. If no start_time is provided, simply take the radius from the first frame (vid.radius[0]) as the initial radius.

In [ ]:
# Get initial radius in meters at time_start
start_index = int(time_start / time_btwn_frames)
R0 = vid.radius[start_index]

When fitting, an initial condition for all variables must be provided as a numpy array. The radius should be provided in the first index of this initial condition. It can be placed elsewhere, but in that case its location must be recorded with the parameter radius_index. As shown in the examples/bezier.ipynb example script, not all processed data will be used in fitting. This is done to speed the fitting process. The percent of the original dataset used can be changed with the pct_data_considered parameter. By default, fitting data is selected uniformly in time. The method of resizing can be changed to selection along arc length by setting resize_with_arc_length to true.

In [ ]:
initial_condition = np.array([R0, 0, 0]) # Initial value in state space of [dR/dt, d(sigma_zz)/dt, d(sigma_rr)/dt].
pct_of_data_to_use = 0.25 # Cut the amount of data used down to increase speed. This is permissible because data has strong correlation as shown above
resize_with_arc_length = False # Use Bezier curves to select which datapoints to keep, rather than a linear arc length method

Every equation has different parameters. As a first guess, the standardly defined Oldroyd-B model is used. This equation has parameters listed as keys below. For each, provide a range of possible values to search across. Output fit parameters will lie within this range. Try to make the range as reasonably small as possible.

Initial guesses for each parameter should be provided as a dictionary if the user has a good intuition for possible fit values. If no guesses are provided, the code will search across ranges at equidistant stations to minimize objective/error. If some guesses are provided, this search will only be performed for parameters without user-provided guesses. The number of stations (divisions of the range to search at) can be increased with the range_sections parameter. Larger range for the same number of stations increases the space between each. Furthermore, increasing the number of stations may drastically increase runtime. The best option is to provide educated guesses for each parameter to avoid this rough searching process.

In [ ]:
# Parameter ranges to vary fitting over
parameter_ranges = {
    'G': (1, 100), # Possible values of linear elastic modulus in SI units
    'gamma': (0, 1), # Possible values of surface tension in SI units
    'eta_S': (0.001, 5), # Possible values of shear viscosity in SI units
    'lamb': (0.001, 1) # Possible values of relaxation time in SI units
}

New guesses and corresponding objectives are printed as iterative guessing proceeds. This can be turned off by setting the verbose flag to false. Fitting may take some time depending on configuration parameters and ranges.

In [ ]:
# Fit to Oldroyd-B model and plot results
vid.fit('oldroyd_b', parameter_ranges, parameter_guesses={}, init_cond=initial_condition, pct_data_considered=pct_of_data_to_use, resize_with_arc_length=resize_with_arc_length, time_start=time_start, time_end=time_end)
start = time.time()
vid.plot_fit(log=False, time_start=time_start, time_end=time_end) # Show plot of fit region alone
end = time.time()
print('Fit equation in ' + str(end - start) + ' seconds')

Above, the fit is displayed over the constricted fitting range. The time range which the plot is across can be changed when the plotting function is called. Below, this is done to show the fit in a more macroscopic context

In [ ]:
# Plot over the entire video time range
vid.plot_fit(log=False, time_start=0, time_end=0.35) # The plotting time range is independent of the fitting range.

Plotting can be done on a logarithmic scale too.

In [ ]:
# Show plot on log scale (which is the default option)
vid.plot_fit(log=True, time_start=time_start, time_end=time_end)

Choosing the correct fitting equation is crucial. Here, the fit is clearly not perfect, Oldroyd-B may have not been the best choice for a model. Let's try a simple linear fit over the same time range instead.

Note, now the names and ranges of parameters must change corresponding to the new equation choice. Also, since a linear fit is not a differential equation, the ode flag should be set to false.

In [ ]:
p = {
    'm': (-1, 0),
    'b': (0, 0.002)
}
ode = False

Since the linear portion of the graph can be easily visualized, educated guesses for each parameters can be provided too. This should speed the fitting process and avoid rough, inaccurate guessing over denoted ranges.

In [ ]:
parameter_guesses = {
    'm': -0.03,
    'b': 0.001
}

In [ ]:
vid.fit('linear', p, ode=ode, parameter_guesses=parameter_guesses, pct_data_considered=pct_of_data_to_use, resize_with_arc_length=resize_with_arc_length, time_start=time_start, time_end=time_end)
vid.plot_fit(log=False, time_start=time_start, time_end=time_end)

The linear fit looks pretty good! We are likely done here.

If none of the standardly-defined equations fit well, a custom equation can be provided. This equation must match the format of standardly-defined equations as shown below. Arguments must be time, y-value, and individual input parameters. Y-value should be a numpy array containing radius at location radius_index (which is defaulted to 0, or first). The output of the function should be the derivative of the y-value according to the equation. If the equation is not a differential equation, set ode to false.

In [ ]:
def mass_dashpot(t, y, k, b):
    # Models the equation: y'' + by' + ky = 0
    # Where k is spring constant, b is friction coefficient
    y_val = y[0]
    dydt_val = y[1]
    output = np.array([dydt_val, -k*y_val - b*dydt_val])
    return output

The name of argument parameters must also be recorded in the same order they are supplied to the function used the equations_args parameter. Ranges, initial condition, and possible guesses should be defined just as they would be with a standardly-defined function.

In [ ]:
equation_args = ['k', 'b']
parameter_ranges = {
    'k': (0, 100),
    'b': (0, 25)
}
initial_condition = np.array([R0, 0])

In [ ]:
vid.fit(mass_dashpot, parameter_ranges, equation_args=equation_args, ode=True, parameter_guesses={}, init_cond=initial_condition, pct_data_considered=pct_of_data_to_use, resize_with_arc_length=resize_with_arc_length, time_start=time_start, time_end=time_end)
vid.plot_fit(log=False, time_start=0, time_end=0.35)

This is not the best fit, but demonstrates how to provide a custom equation!